In [1]:
from PIL import Image


img = Image.open(r'F:\work\python\clone\2d\Pytorch-UNet\data\imgs\sample\Image_20240622155143140_OUT.png')
print(img)

<PIL.PngImagePlugin.PngImageFile image mode=1 size=3072x2048 at 0x1457B1B0CD0>


In [5]:
import cv2

img1=cv2.imread(r'F:\work\python\clone\2d\Pytorch-UNet\data\imgs\sample\barsample1_OUT.png')
print(img1)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from utils.data_loading import BasicDataset
from unet import UNet
from utils.utils import plot_img_and_mask

def predict_img(net, full_img, device, scale_factor=1, out_threshold=0.5):
    net.eval()
    img = torch.from_numpy(BasicDataset.preprocess(None, full_img, scale_factor, is_mask=False))
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)
    logging.info(f'Preprocessed image shape: {img.shape}')

    with torch.no_grad():
        output = net(img).cpu()
        output = F.interpolate(output, (full_img.size[1], full_img.size[0]), mode='bilinear')
        if net.n_classes > 1:
            mask = output.argmax(dim=1)
        else:
            mask = torch.sigmoid(output) > out_threshold
    logging.info(f'Predicted mask unique values: {torch.unique(mask)}')
    return mask[0].long().squeeze().numpy()

def get_args():
    parser = argparse.ArgumentParser(description='Predict mask from input images')
    parser.add_argument('--model', '-m', default='2d/Pytorch-UNet/model/checkpoint_epoch10.pth', metavar='FILE',
                        help='Specify the file in which the model is stored')
    parser.add_argument('--input', '-i', metavar='INPUT', nargs='+', help='Filenames of input images')
    parser.add_argument('--output', '-o', metavar='OUTPUT', nargs='+', help='Filenames of output images')
    parser.add_argument('--viz', '-v', action='store_true',
                        help='Visualize the images as they are processed')
    parser.add_argument('--no-save', '-n', action='store_true', help='Do not save the output mask')
    parser.add_argument('--mask-threshold', '-t', type=float, default=0.5,
                        help='Minimum probability value to consider a mask pixel white')
    parser.add_argument('--scale', '-s', type=float, default=0.5,
                        help='Scale factor for the input images')
    parser.add_argument('--bilinear', action='store_true', default=False, help='Use bilinear upsampling')
    parser.add_argument('--classes', '-c', type=int, default=2, help='Number of classes')

    args = parser.parse_args()

    if args.input is None:
        args.input = [r'F:\work\python\clone\2d\Pytorch-UNet\data\imgs\barsample.jpg']

    return args

def get_output_filenames(args):
    def _generate_name(fn):
        return f'{os.path.splitext(fn)[0]}_OUT.png'
    return args.output or list(map(_generate_name, args.input))

def mask_to_image(mask: np.ndarray, mask_values):
    logging.info(f'Mask shape: {mask.shape}, unique values: {np.unique(mask)}')
    if isinstance(mask_values[0], list):
        out = np.zeros((mask.shape[-2], mask.shape[-1], len(mask_values[0])), dtype=np.uint8)
    elif mask_values == [0, 1]:
        out = np.zeros((mask.shape[-2], mask.shape[-1]), dtype=bool)
    else:
        out = np.zeros((mask.shape[-2], mask.shape[-1]), dtype=np.uint8)

    if mask.ndim == 3:
        mask = np.argmax(mask, axis=0)

    for i, v in enumerate(mask_values):
        out[mask == i] = v

    return Image.fromarray(out)

if __name__ == '__main__':
    args = get_args()

    img_dir = r"F:\work\python\clone\2d\Pytorch-UNet\data\imgs\sample"
    fname = os.listdir(img_dir)
    for name in fname:
        fname2 = os.path.join(img_dir, name)
        if args.input is None:
            args.input = [fname2]
        else:
            args.input.append(fname2)

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

    in_files = args.input
    out_files = get_output_filenames(args)

    net = UNet(n_channels=3, n_classes=args.classes, bilinear=args.bilinear)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Loading model {args.model}')
    logging.info(f'Using device {device}')

    net.to(device=device)
    state_dict = torch.load(args.model, map_location=device)
    logging.info(f'Model loaded with parameters: {list(net.parameters())[0].data}')
    mask_values = state_dict.pop('mask_values', [0, 1])
    net.load_state_dict(state_dict)

    logging.info('Model loaded!')

    for i, filename in enumerate(in_files):
        logging.info(f'Predicting image {filename} ...')
        img = Image.open(filename)

        mask = predict_img(net=net,
                           full_img=img,
                           scale_factor=args.scale,
                           out_threshold=args.mask_threshold,
                           device=device)



usage: ipykernel_launcher.py [-h] [--model FILE] [--input INPUT [INPUT ...]]
                             [--output OUTPUT [OUTPUT ...]] [--viz]
                             [--no-save] [--mask-threshold MASK_THRESHOLD]
                             [--scale SCALE] [--bilinear] [--classes CLASSES]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Liminghui\AppData\Roaming\jupyter\runtime\kernel-b15d4986-7d81-481e-a485-73882b6175e1.json


SystemExit: 2

F:\work\environment\envs\bar\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
(New-Object Net.WebClient).DownloadFile("https://dl.google.com/dl/cloudsdk/channels/rapid/GoogleCloudSDKInstaller.exe", "$env:Temp\GoogleCloudSDKInstaller.exe")

& $env:Temp\GoogleCloudSDKInstaller.exe
    

SyntaxError: invalid syntax. Perhaps you forgot a comma? (951371657.py, line 1)

In [5]:
from PIL import Image
import numpy as np
import os

def get_max_gray_value(image_path):
    # 打开图像
    img = Image.open(image_path).convert('L')  # 转换为灰度图像

    # 转换为numpy数组
    img_array = np.array(img)

    # 获取最高灰度值
    max_gray_value = img_array.max()
    print(f"最高灰度值: {max_gray_value}")

    return img_array, img

def replace_gray_value(img_array, old_value, new_value):
    # 将灰度值为old_value的像素转换为new_value
    img_array[img_array == old_value] = new_value

    # 转换为图像
    new_img = Image.fromarray(img_array)

    return new_img

if __name__ == "__main__":
    # 替换为您的图片路径
    # image_path = 'path_to_your_image.png'
    image_path = r"F:\work\dataset\rebar2D\train2\mask"
    output_dir = r"F:\work\dataset\rebar2D\train2\TEMP"
    for filename in os.listdir(image_path):
        if filename.endswith(".png"):
            input_path = os.path.join(image_path, filename)
            # 获取最高灰度值并返回图像数组
            img_array, img = get_max_gray_value(input_path)

            # 将灰度值为1的像素转换为255
            new_img = replace_gray_value(img_array, 1, 255)

            # 保存新图像
            new_image_path = os.path.join(output_dir, filename)
            new_img.save(new_image_path)
            print(f"新图像已保存到: {new_image_path}")
    # # 获取最高灰度值并返回图像数组
    # img_array, img = get_max_gray_value("/kaggle/input/rebar2d/train2/mask/Image_20240622152006003_mask.png")
    # 
    # # 将灰度值为1的像素转换为255
    # new_img = replace_gray_value(img_array, 1, 255)
    # 
    # # 保存新图像
    # new_image_path = 'path_to_save_new_image.png'
    # new_img.save(new_image_path)
    # print(f"新图像已保存到: {new_image_path}")


最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152006003_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152022155_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152027051_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152032646_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152038023_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152043066_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152049220_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152054365_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152100108_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152105224_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152110857_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\t